In [9]:
import pandas as pd
import matplotlib as plt
from pandas import ExcelWriter
import os
import openpyxl
import locale
locale.setlocale(locale.LC_ALL,('es_ES','UTF-8'))
año=2023
TC={ '1.2019':3.379,
     '2.2019':3.335,
     '3.2019':3.305,
     '4.2019':3.312,
     '5.2019':3.312,
     '6.2019':3.370,
     '7.2019':3.291,
     '8.2019':3.310,
     '9.2019':3.405,
     '10.2019':3.385,
     '11.2019':3.336,
     '12.2019':3.403,   
    '1.2022':3.998,
     '2.2022':3.846,
     '3.2022':3.759,
     '4.2022':3.701,
     '5.2022':3.838,
     '6.2022':3.707,
     '7.2022':3.830,
     '8.2022':3.925,
     '9.2022':3.847,
     '10.2022':3.984,
     '11.2022':3.993,
     '12.2022':3.854,
     '1.2023':3.820,
     '2.2023':3.851}
input_loc='//192.168.10.250/finanzas_produccion/FINANZAS_PRODUCCION/Fernando/BASE DE DATOS/DATA DIARIA/DATA BRUTA/'+str(año)+'/'
output_loc='//192.168.10.250/finanzas_produccion/FINANZAS_PRODUCCION/Fernando/BASE DE DATOS/DATA DIARIA/DATA LIMPIA/'+str(año)+'/'

In [10]:
filelist=os.listdir(input_loc)
for files in filelist:
   if files.endswith(".xlsx"):
      file_date=files.replace('.xlsx', '')
      file_month=int(file_date[3:5])
      file_year=int(file_date[-4:])
      file_day=int(file_date[:2])
      # SELECCIÓN DE DÍAS A ACTUALIZAR EN LA BASE DE DATOS
      if file_month==2:
         df=pd.read_excel(input_loc+files)
         df.columns=df.iloc[0]
         df=df.drop(0)
         df=df.drop(df.tail(1).index)
         df['FECHA']=file_date
         FECHA=df.pop('FECHA')
         df.insert(0,'FECHA',FECHA)
         df['FECHA']=pd.to_datetime(df['FECHA'], format="%d.%m.%Y")
         df[['Marca','Tipo','Version']]=df['Marca/Tipo/Version'].str.split(pat='/',n=2,expand=True)
         df['Coin($/Maq-dia)']=(df['Dif. ElecIn']/TC.get(str(file_month)+'.'+str(file_year),3.9))
         df['Netwin Real($/Maq-dia)']=(df['Neto Elec.']/TC.get(str(file_month)+'.'+str(file_year),3.9))
         df['Netwin Teorico($/Maq-dia)']=df['Coin($/Maq-dia)']*0.06
         df=df.drop(['N°','Prog','Piso','Bill','Marca/Tipo/Version'],axis=1)
         df.to_excel(output_loc+files,'DATA',index=False)